In [1]:
import math
import numpy as np

In [67]:
class Node:
   def __init__(self, data):
      self.children = {}
      self.data = data

   def add_child(self, edge, node):
      self.children[edge] = node

   def print(self):
      print(self.data)

   def print_helper(self, edge, level):
      if len(self.children) == 0:
         print(f'[{level}]: {edge} {self.data} ')

      else:
         print(f'[{level}]: {edge} {self.data} ')
         for i in self.children:
            self.children[i].print_helper(i, level+1)

   def print_tree(self):
      self.print_helper('', 0)


In [7]:
train_X = np.array([['-', '+', '+', '-'],
                    ['+', '+', '+', '+'],
                    ['-', '+', '-', '+'],
                    ['-', '-', '+', '-'],
                    ['+', '+', '-', '+']])

test_X = np.array([['+', '-', '-'],
                   ['-', '-', '-'],
                   ['+', '-', '+']])

DTree(examples, features) returns a tree
  If all examples are in one category, return a leaf node with that category label.
  Else if the set of features is empty, return a leaf node with the category label that
      is the most common in examples.
  Else pick a feature F and create a node R for it
      For each possible value vi of F:
         Add an out-going edge E to node R labeled with the value vi.
         Let examplesi be the subset of examples that have value vi for F
	      If examplesi is empty
            then attach a leaf node to edge E labeled with the category that
               is the most common in examples.
         else call DTree(examplesi , features – {F}) and attach the resulting
            tree as the subtree under edge E.
      Return the subtree rooted at R.       


In [54]:
def one_label(examples):
   label = None
   for example_i in examples:
      if label == None:
         label = example_i[examples.shape[1] - 1]
      else:
         if example_i[examples.shape[1] - 1] != label:
            return False, None
   return True, label

def count_positive_examples(examples):
   positives = 0
   for example_i in examples:
      if example_i[examples.shape[1] - 1] == '+':
         positives += 1
   return positives, examples.shape[0] - positives

def calc_entropy_S(examples):
   p_1, p_0 = count_positive_examples(examples)
   entropy_1 = -p_1 * math.log(p_1, 2) if p_1 > 0 else 0
   entropy_0 = -p_0 * math.log(p_0, 2) if p_0 > 0 else 0
   return entropy_1 + entropy_0

def decision_tree(examples, features):
   all_one_label, temp_label = one_label(examples)
   if all_one_label:
      return Node(temp_label)

   elif len(features) == 0:
      pos, neg = count_positive_examples(examples)
      return Node('+' if pos >= neg else '-')

   else:
      entropy_S = calc_entropy_S(examples)
      picked_F, max_gain = 0, -math.inf
      for F in features:
         s_F = np.column_stack([examples[:,F-1], examples[:,examples.shape[1] - 1]])
         pos_vals = s_F[np.where(s_F[:,0] == '+')]
         neg_vals = s_F[np.where(s_F[:,0] == '-')]
         F_gain = entropy_S - ((pos_vals.shape[0] / examples.shape[0]) * calc_entropy_S(pos_vals) + (neg_vals.shape[0] / examples.shape[0]) * calc_entropy_S(neg_vals))
         if F_gain > max_gain:
            picked_F = F
            max_gain = F_gain
      
      root = Node(picked_F)

      for val_i in ['+', '-']:
         examples_i = examples[np.where(examples[:,picked_F-1] == val_i)]
         if examples_i.shape[0] == 0:
            pos, neg = count_positive_examples(examples)
            root.add_child('+' if pos >= neg else '-', picked_F)
         else:
            root.add_child(val_i, decision_tree(examples_i, features[features != picked_F]))

      return root


In [68]:
dt = decision_tree(train_X, np.array(range(1, train_X.shape[1])))
dt.print_tree()

[0]:  2 
[1]: + 1 
[2]: + + 
[2]: - 3 
[3]: + - 
[3]: - + 
[1]: - - 


The final hypothesis looks like (click on cell to view tree):
            2
         [+]  [-]
      1           -
   [+]  [-]
+          3
       [+]   [-]
      -          +

For the three test examples:
   - (+, -, -) = -
   - (-, -, -) = -
   - (+, -, +) = -